In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import subprocess
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/School/Term 8/AI Project/WELFake_Dataset.csv')
# df = pd.read_csv('/content/WELFake_Dataset.csv')

Data Preprocessing

In [4]:
df = df.dropna()
df['title and text'] = df['title'] + ' ' + df['text']
display(df)

,Unnamed: 0,title,text,label,title and text
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,About Time! Christian Group Sues Amazon and SP...
...,...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,Russians steal research on Trump in hack of U....
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,WATCH: Giuliani Demands That Democrats Apolog...
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,Migrants Refuse To Leave Train At Refugee Camp...
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,Trump tussle gives unpopular Mexican leader mu...


In [5]:
X = df['title and text'].values[:5707]
y = df['label'].values[:5707]
print(len(X))

5707


Entire dataset now split into X and y, where X is the append of both title and text while the y are the labels

In [6]:
X = df['title and text'].values
y = df['label'].values

Tokeniser: Each integer in the sequence represents the index of the corresponding word in the vocabulary (word index) built by the Tokenizer. These integer indices are just assigned during tokenisation and they do not capture semantic relationships between words.

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#convert text to vector form (convert text into vector form no semantic meaning is covered)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X) # assigns each word an unique vector
sequences = tokenizer.texts_to_sequences(X) # this is list of lists where [[doc1],[doc2]] in doc1=[4,6,8,3,6] list representing teh integers index in the tokeniser dictionary



In [8]:
# Remove stopwords
import nltk
from nltk.corpus import stopwords

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
filtered_sequences = []
for seq in sequences:
    filtered_seq = [word for word in seq if word not in stopwords]
    filtered_sequences.append(filtered_seq)

# Pad sequences to make them uniform in length
MAX_SEQUENCE_LENGTH = 1000
# make all the sequence same length(seq means each text row)
padded_sequences = pad_sequences(filtered_sequences, maxlen=MAX_SEQUENCE_LENGTH)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
print(len(tokenizer.word_index))

330837


In [10]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# Further split the remaining data (20%) into validation and testing sets (50% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

dont need to run below until ensemble

# LSTM Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adagrad

model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))

#purpose to complie model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])

#start training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluation on unseen data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
1789/1789 [==============================] - 280s 154ms/step - loss: 0.2791 - accuracy: 0.8844 - val_loss: 0.1547 - val_accuracy: 0.9427
Epoch 2/10
1789/1789 [==============================] - 162s 90ms/step - loss: 0.0865 - accuracy: 0.9696 - val_loss: 0.0728 - val_accuracy: 0.9736
Epoch 3/10
1789/1789 [==============================] - 139s 78ms/step - loss: 0.0288 - accuracy: 0.9903 - val_loss: 0.0700 - val_accuracy: 0.9757
Epoch 4/10
1789/1789 [==============================] - 127s 71ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.0759 - val_accuracy: 0.9803
Epoch 5/10
1789/1789 [==============================] - 122s 68ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.0915 - val_accuracy: 0.9797
Epoch 6/10
1789/1789 [==============================] - 121s 67ms/step - loss: 0.0030 - accuracy: 0.9992 - val_loss: 0.0803 - val_accuracy: 0.9762
Epoch 7/10
1789/1789 [==============================] - 118s 66ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 

In [12]:
# Save the model
# model.save("/content/drive/MyDrive/School/Term 8/AI Project/lstm_model.pth")
model.save("/content/drive/MyDrive/AI Project/my_model_adagrad_sigmoid_256.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# GRU Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, SpatialDropout1D

# Define model architecture
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
gru_units = 64

# gru_model = Sequential()
# gru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=MAX_SEQUENCE_LENGTH))
# gru_model.add(SpatialDropout1D(0.2))
# gru_model.add(GRU(gru_units, dropout=0.2, recurrent_dropout=0.2))
# gru_model.add(Dense(1, activation='sigmoid'))

gru_model = Sequential()
gru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=MAX_SEQUENCE_LENGTH))
gru_model.add(GRU(gru_units, dropout=0.5))
gru_model.add(Dense(1, activation='sigmoid'))

# Compile model
gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
print(gru_model.summary())

# Train model
history = gru_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate model
loss, accuracy = gru_model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1000, 100)         33083800  
                                                                 
 gru (GRU)                   (None, 64)                31872     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33115737 (126.33 MB)
Trainable params: 33115737 (126.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
1789/1789 [==============================] - 194s 106ms/step - loss: 0.1985 - accuracy: 0.9250 - val_loss: 0.1583 - val_accuracy: 0.9403
Epoch 2/10
1789/1789 [==============================] - 112s 62ms/step - loss: 0.1122 - accuracy: 0.9613 - val_loss: 0.11

In [18]:
gru_model.save("/content/drive/MyDrive/School/Term 8/AI Project/gru_model.h5")
# gru_model.save("/content/drive/MyDrive/School/Term 8/AI Project/gru_model.pth")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Define model architecture
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 100

# Define CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=MAX_SEQUENCE_LENGTH))
cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
1789/1789 [==============================] - 157s 86ms/step - loss: 0.0916 - accuracy: 0.9621 - val_loss: 0.0414 - val_accuracy: 0.9855
Epoch 2/10
1789/1789 [==============================] - 71s 40ms/step - loss: 0.0123 - accuracy: 0.9959 - val_loss: 0.0441 - val_accuracy: 0.9854
Epoch 3/10
1789/1789 [==============================] - 49s 27ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0558 - val_accuracy: 0.9864
Epoch 4/10
1789/1789 [==============================] - 42s 24ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0993 - val_accuracy: 0.9788
Epoch 5/10
1789/1789 [==============================] - 40s 23ms/step - loss: 0.0042 - accuracy: 0.9983 - val_loss: 0.0997 - val_accuracy: 0.9789
Epoch 6/10
1789/1789 [==============================] - 42s 23ms/step - loss: 9.1291e-04 - accuracy: 0.9997 - val_loss: 0.0685 - val_accuracy: 0.9859
Epoch 7/10
1789/1789 [==============================] - 36s 20ms/step - loss: 0.0022 - accuracy: 0.9992 - val_loss: 0.0

In [ ]:
cnn_model.save("/content/drive/MyDrive/School/Term 8/AI Project/cnn_model.h5")
cnn_model.save("/content/drive/MyDrive/School/Term 8/AI Project/cnn_model.pth")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Once all the CNN, GRU and LSTM models are saved, don't need to run the above again

# Stacking ensemble model with base-models CNN, GRU and LSTM

In [19]:
from tensorflow.keras.models import Sequential,load_model

# load the models from google drive
cnn_f = "/content/drive/MyDrive/School/Term 8/AI Project/cnn_model.h5"
gru_f = "/content/drive/MyDrive/School/Term 8/AI Project/gru_model.h5"
lstm_f = "/content/drive/MyDrive/School/Term 8/AI Project/my_model_adagrad_sigmoid_256.h5"
# lstm_f = "/content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_john.h5"

cnn_m = load_model(cnn_f)
gru_m = load_model(gru_f)
lstm_m = load_model(lstm_f)

members = []
members.append(cnn_m)
members.append(gru_m)
members.append(lstm_m)

In [20]:
from numpy import dstack
from sklearn.linear_model import LogisticRegression

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(base_members, inputX):
	stackX = None
	for model in base_members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, base_members, probabilities]
		if stackX is None:
			stackX = yhat #
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, base_members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# meta learner model
	model = LogisticRegression()
	model.fit(stackedX, inputy)
	return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [21]:
ensemble_model = fit_stacked_model(members, X_test, y_test)

In [22]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# evaluate ensemble model on test set
ensemble_predictions = stacked_prediction(members, ensemble_model, X_val)

# Evaluate performance
accuracy = accuracy_score(y_val, ensemble_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.988118535085267
